# Chatbot with transformer

## Rubric

### 1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?

챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.

### 2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?	

과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.

### 3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?	

주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.

In [1]:
!pip install gensim==3.8.1 # wv을 쓰기위해서 gensim 버전 다운그레이드

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import re
import os
import io
import time
import random

from konlpy.tag import Mecab
import tensorflow as tf

import seaborn

In [3]:
data = pd.read_csv('/aiffel/aiffel/transformer_chatbot/data/ChatbotData.csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
len(data) # 3배 증강 필요

11823

In [5]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ0-9]+", " ", sentence)
    sentence = sentence.strip()
    
    return sentence

In [6]:
questions = [preprocess_sentence(q) for q in data['Q']]
answers = [preprocess_sentence(a) for a in data['A']]

    
print('questions data size :', len(questions))
print('answers data size :', len(answers))
print()
for i in range(10):
    print('Q :', questions[i])
    print('A :', answers[i])    
    print()

questions data size : 11823
answers data size : 11823

Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : ppl 심하네
A : 눈살이 찌푸려지죠.

Q : sd카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : sd카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : sns 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : sns 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : sns 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



In [7]:
def tokenize(corpus, vocab_size=50000):
    mecab = Mecab()
    morph = [" ".join(mecab.morphs(sen)) for sen in corpus]
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=vocab_size)
    tokenizer.fit_on_texts(morph)

    tensor = tokenizer.texts_to_sequences(morph)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer, morph


In [8]:
def tokenize_answer(corpus, vocab_size=50000):
    mecab = Mecab()

    # 형태소 분석
    morph = [" ".join(mecab.morphs(sen)) for sen in corpus]

    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=vocab_size)
    tokenizer.fit_on_texts(morph)

    # 토큰화 사전에 <start>와 <end> 토큰 추가
    tokenizer.word_index['<start>'] = len(tokenizer.word_index) + 1
    tokenizer.word_index['<end>'] = len(tokenizer.word_index) + 2

    # `index_word` dictionary에 <start>와 <end> 토큰 추가
    tokenizer.index_word[tokenizer.word_index['<start>']] = '<start>'
    tokenizer.index_word[tokenizer.word_index['<end>']] = '<end>'

    # <start>와 <end> 토큰을 추가하여 토큰화
    tensor = tokenizer.texts_to_sequences(["<start> " + m + " <end>" for m in morph])
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer, morph

In [9]:
from gensim.models.word2vec import Word2Vec

# Word2Vec 모델을 불러옵니다.
w2v_path = '/aiffel/data/ko.bin'  
wv_model = Word2Vec.load(w2v_path)

def lexical_sub(sentence, word2vec):
    toks = sentence.split()
    
    # 랜덤한 토큰 선택
    _from = random.choice(toks)
    
    try:
        _to = word2vec.wv.most_similar(_from)[0][0]  # wv으로 유의어 변환
        # 선택된 토큰을 유의어로 변경
        toks[toks.index(_from)] = _to
    except:  # 단어장에 없는 단어
        pass

    return ' '.join(toks)

def lexical_sub_0_33(sentence, word2vec):
    toks = sentence.split()
    
    for idx, _from in enumerate(toks):
        if (idx + 1) % 3 == 0:  # 토큰의 3의 배수마다
            try:
                _to = word2vec.wv.most_similar(_from)[0][0] # wv으로 유의어 변환 
                toks[idx] = _to
            except:   # 단어장에 없는 단어
                continue

    return ' '.join(toks)

def lexical_sub_first_token(sentence, word2vec):
    toks = sentence.split()
    
    # 첫 번째 토큰 선택
    _from = toks[0]
    
    try:
        _to = word2vec.wv.most_similar(_from)[0][0]  # wv으로 유의어 변환
        # 첫 번째 토큰을 유의어로 변경
        toks[0] = _to
    except:  # 단어장에 없는 단어 or 모델이 해당 토큰에 대한 유사어를 찾지 못할 경우
        pass

    return ' '.join(toks)


In [10]:
# questions와 answers에 대해 lexical_sub 함수로 증강
questions_augmented = [lexical_sub(question, wv_model) for question in questions]
answers_augmented = [lexical_sub(answer, wv_model) for answer in answers]

# questions와 answers에 대해 lexical_sub 함수로 증강(한번더)
questions_augmented2 = [lexical_sub(question, wv_model) for question in questions]
answers_augmented2 = [lexical_sub(answer, wv_model) for answer in answers]

# questions와 answers에 대해 lexical_sub_0_33 함수로 증강
questions_augmented_0_33 = [lexical_sub_0_33(question, wv_model) for question in questions]
answers_augmented_0_33 = [lexical_sub_0_33(answer, wv_model) for answer in answers]

# questions와 answers에 대해 lexical_sub 함수로 증강
questions_augmented_first = [lexical_sub_first_token(question, wv_model) for question in questions]
answers_augmented_first = [lexical_sub_first_token(answer, wv_model) for answer in answers]

# 원본 questions와 answers에 증강된 데이터 추가
questions.extend(questions_augmented)
questions.extend(questions_augmented2)
questions.extend(questions_augmented_0_33)
questions.extend(questions_augmented_first)

answers.extend(answers_augmented)
answers.extend(answers_augmented2)
answers.extend(answers_augmented_0_33)
answers.extend(answers_augmented_first)

print('Total questions data size after augmentation:', len(questions))
print('Total answers data size after augmentation:', len(answers))

Total questions data size after augmentation: 59115
Total answers data size after augmentation: 59115


In [11]:
for i in range(10):
    print('aug Q :', questions_augmented[i])
    print('aug A :', answers_augmented[i])
    print()
    
for i in range(10):
    print('0.33aug Q :', questions_augmented_0_33[i])
    print('0.33aug A :', answers_augmented_0_33[i])
    print()
    

aug Q : 12시 땡!
aug A : 하루가 각기 가네요.

aug Q : 1지망 학교의 떨어졌어
aug A : 위로해 드립니다.

aug Q : 3박4일 놀러가고 싶다
aug A : 여행은 항상 좋죠.

aug Q : 3박4일 정도 놀러가고 싶다
aug A : 여행은 언제나 좋죠.

aug Q : ppl 심하네
aug A : 눈살이 찌푸려지죠.

aug Q : sd카드 망가졌어
aug A : 다시 새로이 사는 게 마음 편해요.

aug Q : sd카드 안돼
aug A : 다시 새로 사는 도록 마음 편해요.

aug Q : sns 맞팔 왜 안하지ㅠㅠ
aug A : 잘 모르고 있을 수도 있어요.

aug Q : sns 시간낭비인 거 아는데 매일 하는 중
aug A : 시간을 정하고 해보세요.

aug Q : sns 시간낭비인데 자꾸 보게됨
aug A : 시간을 정하고 해보세요.

0.33aug Q : 12시 땡!
0.33aug A : 하루가 또 가네요.

0.33aug Q : 1지망 학교 떨어졌어
0.33aug A : 위로해 드립니다.

0.33aug Q : 3박4일 놀러가고 싶다
0.33aug A : 여행은 언제나 좋죠.

0.33aug Q : 3박4일 정도 놀러가고 싶다
0.33aug A : 여행은 언제나 좋죠.

0.33aug Q : ppl 심하네
0.33aug A : 눈살이 찌푸려지죠.

0.33aug Q : sd카드 망가졌어
0.33aug A : 다시 새로 사는 게 마음 편해요.

0.33aug Q : sd카드 안돼
0.33aug A : 다시 새로 사는 게 마음 편해요.

0.33aug Q : sns 맞팔 과연 안하지ㅠㅠ
0.33aug A : 잘 모르고 있을 수도 있어요.

0.33aug Q : sns 시간낭비인 것 아는데 매일 하는 중
0.33aug A : 시간을 정하고 해보세요.

0.33aug Q : sns 시간낭비인데 그럼 보게됨
0.33aug A : 시간을 정하고 해보세요.



In [12]:
# questions와 answers를 튜플로 묶기
combined = list(zip(questions, answers))

# 중복 제거
unique_combined = list(set(combined))

# 다시 questions와 answers로 분리
questions_unique, answers_unique = zip(*unique_combined)

print('Total unique questions data size:', len(questions_unique))
print('Total unique answers data size:', len(answers_unique)) #대략 3배 증강


Total unique questions data size: 27649
Total unique answers data size: 27649


In [13]:
# 질문 토큰화
questions_tensor, questions_tokenizer, _ = tokenize(questions_unique)

# 답변 토큰화
answers_tensor, answers_tokenizer, _ = tokenize_answer(answers_unique)

# 첫 번째 질문 텐서 출력
print("First question tensor:", questions_tensor[0])

# 첫 번째 질문 텐서를 원래 문장으로 복원
decoded_question = " ".join([questions_tokenizer.index_word[idx] for idx in questions_tensor[0] if idx != 0]) # 0은 padding 토큰
print("Decoded question:", decoded_question)

# 첫 번째 답변 텐서 출력
print("\nFirst answer tensor:", answers_tensor[0])

# 첫 번째 답변 텐서를 원래 문장으로 복원
decoded_answer = " ".join([answers_tokenizer.index_word[idx] for idx in answers_tensor[0] if idx != 0]) # 0은 padding 토큰
print("Decoded answer:", decoded_answer)

First question tensor: [ 10   8   2   1  20  16  18  36   7 164 857  13   3   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Decoded question: 좋 아 하 는 사람 있 는데 내 가 조금 소심 해 .

First answer tensor: [4385  343  288 1238  395   27    9    6    8   12    1 4387    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]
Decoded answer: <start> 달라지 긴 원한다면 때때 수 있 을 거 예요 . <end>


In [15]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(questions_tensor, answers_tensor, test_size=0.01)

print("enc_train size:", len(enc_train), "enc_val size:", len(enc_val))
print("dec_train size:", len(dec_train), "dec_val size:", len(dec_val))

enc_train size: 27372 enc_val size: 277
dec_train size: 27372 dec_val size: 277


In [16]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [17]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [18]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

In [19]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

In [20]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [22]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [23]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [24]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [25]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
    
learning_rate = LearningRateScheduler(512)
oprimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [26]:
transformer = Transformer(
    n_layers=4,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=len(enc_train),
    tgt_vocab_size=len(dec_train),
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)


In [27]:
d_model=512
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [29]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [30]:
from tqdm import tqdm_notebook 

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)
    
    # 입력 문장을 토큰화하고, 시작 및 종료 토큰을 추가합니다.
    encoded_sentence = src_tokenizer.texts_to_sequences([sentence])[0]
    sentence = tf.expand_dims(encoded_sentence, axis=0)

    # 디코더의 시작 토큰
    output = tf.expand_dims([tgt_tokenizer.word_index['<start>']], 0)

    # 디코더의 예측 시작
    for i in range(40):
        enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(sentence, output)
        predictions, _, _, _ = model(sentence, 
                                     output,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)
        
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        if tf.equal(predicted_id, tgt_tokenizer.word_index['<end>']):
            break
        output = tf.concat([output, predicted_id], axis=-1)

    result = tf.squeeze(output).numpy()
    if np.isscalar(result):
        result = [result]
    
    return ' '.join([tgt_tokenizer.index_word[i] for i in result if i in tgt_tokenizer.index_word and i not in [tgt_tokenizer.word_index['<start>'], tgt_tokenizer.word_index['<end>']]])



In [31]:
BATCH_SIZE = 64
EPOCHS = 30


for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                   dec_train[idx:idx+BATCH_SIZE],
                   transformer,
                   optimizer)
        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    # 에포크가 끝날 때마다 예문으로 예측
    print("\nSample Predictions after epoch {}\n".format(epoch+1))
    sample_sentences = ["안녕하세요?", "오늘 날씨 어때?", "뭐 먹을까?", "영화 볼래?", "휴일에는 뭐하는게 좋을까?", "최근에 본 좋은 영화 추천해줄래?", "오늘의 점심 메뉴는 뭐가 좋을까?"]
    
    for sentence in sample_sentences:
        prediction = evaluate(sentence, transformer, questions_tokenizer, answers_tokenizer)
        print(f'Input: {sentence}')
        print(f'Output: {prediction}\n')


/tmp/ipykernel_418/2459443605.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm_notebook(idx_list)


  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after epoch 1

Input: 안녕하세요?
Output: 이제 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 이 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 은 지금 은 지금 .

Input: 오늘 날씨 어때?
Output: 저 이 에요 .

Input: 뭐 먹을까?
Output: 연락 이 이 이 이 에요 .

Input: 영화 볼래?
Output: 마음 이 에요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 이제 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 이 지금 지금 지금 지금 지금 지금 지금 지금 지금 지금 은 지금 은 지금 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 지금 이 에요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 많이 세요 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after epoch 2

Input: 안녕하세요?
Output: 당신 의 짐 이 복잡 하 지 않 았 나 봐요 .

Input: 오늘 날씨 어때?
Output: 기분 이 라도 마시 세요 .

Input: 뭐 먹을까?
Output: 저 는 게 좋 을 거 같 아요 .

Input: 영화 볼래?
Output: 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화 영화

Input: 휴일에는 뭐하는게 좋을까?
Output: 당신 의 짐 이 복잡 하 지 않 았 나 봐요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 영화 영화 보 세요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 같이 가요 .



...


Sample Predictions after epoch 28

Input: 안녕하세요?
Output: 무엇 이 든 손 에 냉정 할 수 있 어요 .

Input: 오늘 날씨 어때?
Output: 날씨 어 플 에 물 어 보 세요 .

Input: 뭐 먹을까?
Output: 음료수 파먹 기 해 보 세요 .

Input: 영화 볼래?
Output: 멀티미디어 영화 가 좋 을 것 같 네요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 무엇 이 든 손 에 냉정 할 수 있 어요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 위로 해 드립니다 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 휴식 도 그럴 거 예요 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after epoch 29

Input: 안녕하세요?
Output: 잠 을 닫 고 있 나 봐요 .

Input: 오늘 날씨 어때?
Output: 오늘 도 셨 나 봐요 .

Input: 뭐 먹을까?
Output: 저 랑 놀 아요 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 잠 을 닫 고 있 나 봐요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 최신 찾 기 어려워 요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 맛있 는 거 드세요 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after epoch 30

Input: 안녕하세요?
Output: 잠 을 깨 요 ! 생명력 내요 ! !

Input: 오늘 날씨 어때?
Output: 쉬 어 가 세요 .

Input: 뭐 먹을까?
Output: 저 랑 놀 아요 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 지만 영화 가 액션 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 잠 을 깨 요 ! 생명력 내요 ! !

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 잠 을 깨 요 ! 생명력

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 맛있 는 거 드세요 .



In [32]:
ADDITIONAL_EPOCHS = 10  # 추가로 학습할 에포크 수

for epoch in range(ADDITIONAL_EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                   dec_train[idx:idx+BATCH_SIZE],
                   transformer,
                   optimizer)
        total_loss += batch_loss

        t.set_description_str('Additional Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    # 에포크가 끝날 때마다 예문으로 예측합니다.
    print("\nSample Predictions after additional epoch {}\n".format(epoch+1))
    
    for sentence in sample_sentences:
        prediction = evaluate(sentence, transformer, questions_tokenizer, answers_tokenizer)
        print(f'Input: {sentence}')
        print(f'Output: {prediction}\n')


/tmp/ipykernel_418/2098457071.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm_notebook(idx_list)


  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after additional epoch 1

Input: 안녕하세요?
Output: 무엇 이 든 는지 모르 겠 어요 .

Input: 오늘 날씨 어때?
Output: 날씨 에 연락 해서 한두 번 웃 어 보 세요 .

Input: 뭐 먹을까?
Output: 저 랑 놀 아요 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 무엇 이 든 는지 모르 겠 어요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 실감 이 안 되 길 바랄게요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 휴식 도 필요 하 죠 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after additional epoch 2

Input: 안녕하세요?
Output: 무엇 이 든 마음 이 든 손 에 도 덜할 거 예요 .

Input: 오늘 날씨 어때?
Output: 언젠간 관리 웃 어 보 세요 .

Input: 뭐 먹을까?
Output: 저 는 말씀 해 보 세요 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 무엇 이 든 마음 이 든 손 에 도 덜할 거 예요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 최신 ?

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 맛있 는 거 드세요 .



...


Sample Predictions after additional epoch 8

Input: 안녕하세요?
Output: 그렇게 고민 하 게 만들 려고 그랬 나 봐요 .

Input: 오늘 날씨 어때?
Output: 내일 도 좋 은 시원 하 시 들 이 말 이 높 드세요 .

Input: 뭐 먹을까?
Output: 멍 때리 고 있 죠 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 그렇게 고민 하 게 만들 려고 그랬 나 봐요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 맛있 는 거 드세요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 휴식 도 떨리 는 날 이 에요 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after additional epoch 9

Input: 안녕하세요?
Output: 아직 순수 하 게 생각 해요 .

Input: 오늘 날씨 어때?
Output: 날씨 에 물 어 보 세요 .

Input: 뭐 먹을까?
Output: 멍 때리 고 있 죠 .

Input: 영화 볼래?
Output: 멀티미디어 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 아직 순수 하 게 생각 해요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 멀티미디어 영화 가 간절 하 겠 네요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 맛있 는 거 드세요 .



  0%|          | 0/428 [00:00<?, ?it/s]


Sample Predictions after additional epoch 10

Input: 안녕하세요?
Output: 계속 어요 .

Input: 오늘 날씨 어때?
Output: 날씨 어 보 세요 .

Input: 뭐 먹을까?
Output: 멍 때리 고 있 죠 .

Input: 영화 볼래?
Output: 최신 영화 가 좋 을 것 같 아요 .

Input: 휴일에는 뭐하는게 좋을까?
Output: 계속 어요 .

Input: 최근에 본 좋은 영화 추천해줄래?
Output: 맛있 을 거 라 생각 해요 .

Input: 오늘의 점심 메뉴는 뭐가 좋을까?
Output: 맛있 게 드세요 .

